# Transformation and Load (With Visualization)

In [9]:
# Installing Libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np

### 1. Read Data

##### 1.1 Read Temperature Data

In [3]:
temp_data=pd.read_excel("variables.xlsx",sheet_name='Temperatura')
temp_data.head(5)


,Time,Temp,Dew
0,2025-02-06 00:00:00,14.04,14.0
1,2025-02-06 00:05:00,14.12,14.3
2,2025-02-06 00:10:00,14.12,13.8
3,2025-02-06 00:15:00,14.20,14.0
4,2025-02-06 00:20:00,14.32,14.0


##### 1.2 Read Humidity Data

In [3]:
hum_data=pd.read_excel("variables.xlsx",sheet_name='Humedad')
hum_data.head(5)

,Time,Hum
0,2025-02-22 11:18:46,70.5
1,2025-02-22 11:33:46,69.5
2,2025-02-22 11:48:46,72.0
3,2025-02-22 12:03:46,75.0
4,2025-02-22 12:18:46,75.0


##### 1.3 Read Radiation Data

In [4]:
rad_data=pd.read_excel("variables.xlsx",sheet_name='Radiacion')
rad_data.head(5)

,Time,PPFD
0,2024-06-28 12:10:00 UTC,73.974609
1,2024-06-28 12:15:00 UTC,103.637695
2,2024-06-28 12:20:00 UTC,130.554199
3,2024-06-28 12:25:00 UTC,185.119629
4,2024-06-28 12:30:00 UTC,216.979980


### 2. Transformation

##### 2.1 Temperature Transformation

2.1.1 Minimum and Maximum temperature

In [4]:
# Set the time variable
temp_data['Time']=pd.to_datetime(temp_data['Time'])

# To get average, Minimum and Maximum
temp_data.set_index('Time',inplace=True)

#Resample by day calculating  min, max and average
temp_data_day=temp_data.resample('D').agg({
    'Temp':['mean','min','max']
})
temp_data_day.head(5)

Temp              
                 mean    min    max
Time                               
2025-02-06  20.008194  11.70  35.54
2025-02-07  20.867222  12.60  38.58
2025-02-08  18.227778  12.38  37.06
2025-02-09  15.745417  12.58  20.68
2025-02-10  15.756944  11.98  24.72

2.1.2 GDD

In [5]:
tbase=10
temp_data_day[('Temp', 'GDD')]=((temp_data_day[('Temp', 'min')]+temp_data_day[('Temp', 'max')])/2)-tbase
temp_data_day[('Temp', 'GDD_cum')]=temp_data_day[('Temp','GDD')].cumsum()
temp_data_day

Temp                             
                 mean    min    max    GDD GDD_cum
Time                                              
2025-02-06  20.008194  11.70  35.54  13.62   13.62
2025-02-07  20.867222  12.60  38.58  15.59   29.21
2025-02-08  18.227778  12.38  37.06  14.72   43.93
2025-02-09  15.745417  12.58  20.68   6.63   50.56
2025-02-10  15.756944  11.98  24.72   8.35   58.91
...               ...    ...    ...    ...     ...
2025-04-26  16.399306  13.30  24.90   9.10  797.45
2025-04-27  19.055903  12.20  33.50  12.85  810.30
2025-04-28  18.084722  12.90  28.20  10.55  820.85
2025-04-29  17.364931  12.00  29.10  10.55  831.40
2025-04-30  15.607634  13.30  23.80   8.55  839.95

[84 rows x 5 columns]

##### 2.2 Humidity Transformation 

In [30]:
# Set the time variable
hum_data['Time']=pd.to_datetime(hum_data['Time'])

# To get average
hum_data.set_index('Time',inplace=True)

#Resample by day calculating mean, min and max
hum_data_day=hum_data.resample('D').agg({
    'Hum':['mean']
})
hum_data_day.head(5)

,Hum
,mean
Time,
2025-02-22,80.647059
2025-02-23,77.416667
2025-02-24,76.234375
2025-02-25,78.072917
2025-02-26,74.338542


##### 2.3 Radiation Transformation

2.3.1 DLI

In [ ]:
# Set the time variable
rad_data['Time']=pd.to_datetime(rad_data['Time'])
# Extract the day of the date 
rad_data['Date'] = rad_data['Time'].dt.date

# Interval --> 5 minutes = 300 seconds
interval=300
# Grouping by day and calculate DLI
dli_diario = rad_data.groupby('Date')['PPFD'].sum() * interval / 1_000_000
dli_diario = dli_diario.reset_index(name='DLI')
dli_diario

,Date,DLI
0,2024-06-28,3.159064
1,2024-06-29,12.205536
2,2024-06-30,10.319843
3,2024-07-01,18.850342
4,2024-07-02,17.173553
...,...,...
259,2025-03-14,8.699194
260,2025-03-15,11.024670
261,2025-03-16,12.838568
262,2025-03-17,12.025854


In [57]:
rad_data.head(2)

,Time,PPFD
0,2024-06-28 12:10:00 UTC,73.974609
1,2024-06-28 12:15:00 UTC,103.637695


### 3. Load and Visualization

##### Temperature Visualizations

In [6]:
fig_temp = go.Figure()

# Now x = df.index
fig_temp.add_trace(go.Scatter(x=temp_data_day.index, y=temp_data_day[('Temp', 'mean')], mode='lines+markers', name='Mean Temp'))
fig_temp.add_trace(go.Scatter(x=temp_data_day.index, y=temp_data_day[('Temp', 'min')], mode='lines+markers', name='Min Temp'))
fig_temp.add_trace(go.Scatter(x=temp_data_day.index, y=temp_data_day[('Temp', 'max')], mode='lines+markers', name='Max Temp'))

# Add titles and labels
fig_temp.update_layout(
    title='Daily Temperature Summary',
    xaxis_title='Time',
    yaxis_title='Temperature (°C)',
    template='plotly_dark'
)

# Show the plot
fig_temp.show()

In [7]:
fig_gdd = go.Figure()

# Now x = df.index
fig_gdd.add_trace(go.Scatter(x=temp_data_day.index, y=temp_data_day[('Temp', 'GDD_cum')], 
                         mode='lines+markers', 
                         name='Mean GDD',
                         line=dict(color='#FFA500'),
                         marker=dict(color='#FFA500')))

# Add titles and labels
fig_gdd.update_layout(
    title='Cumulative GDD Summary per day',
    xaxis_title='Time',
    yaxis_title='GDD (C°)',
    template='plotly_dark'
)

# Show the plot
fig_gdd.show()

In [8]:
# Export temperature plots
## Minimum, Maximum and Mean temperature
fig_temp.write_html("/Users/jorgeandresjolahernandez/Desktop/Produccion_Hortalizas/tomato_climate_project/static/visualizations/temp_day.html",full_html=False,include_plotlyjs='cdn')
fig_gdd.write_html("/Users/jorgeandresjolahernandez/Desktop/Produccion_Hortalizas/tomato_climate_project/static/visualizations/gdd_cum.html")


##### Humidity Visualizations

In [10]:
fig_hum = go.Figure()

# Now x = df.index
fig_hum.add_trace(go.Scatter(x=hum_data_day.index, y=hum_data_day[('Hum', 'mean')], 
                         mode='lines+markers', 
                         name='Mean Hum',
                         line=dict(color='#CCCC00'),
                         marker=dict(color='#CCCC00')))

# Add titles and labels
fig_hum.update_layout(
    title='Daily Humidity Summary',
    xaxis_title='Time',
    yaxis_title='Humidity (%rh)',
    template='plotly_dark'
)

# Show the plot
fig_hum.show()

NameError: name 'hum_data_day' is not defined

In [54]:
# Export humidity plots
fig_hum.write_html("/Users/jorgeandresjolahernandez/Desktop/Produccion_Hortalizas/tomato_climate_project/static/visualizations/hum_day.html")

##### Radation Visualizations

In [63]:
fig_dli = go.Figure()

# Now x = df.index
fig_dli.add_trace(go.Scatter(x=dli_diario['Date'], y=dli_diario['DLI'], 
                         mode='lines+markers', 
                         name='DLI',
                         line=dict(color='#8B4513'),
                         marker=dict(color='#8B4513')))

# Add titles and labels
fig_dli.update_layout(
    title='Daily DLI Summary',
    xaxis_title='Time',
    yaxis_title='DLI (mol/m²/día)',
    template='plotly_dark'
)

# Show the plot
fig_dli.show()

In [64]:
# Export Radiation plots
fig_dli.write_html("/Users/jorgeandresjolahernandez/Desktop/Produccion_Hortalizas/tomato_climate_project/static/visualizations/dli_day.html")